In [1]:
!pip install benchmark_functions
!pip install scipy

In [2]:
import random
from math import log2, ceil
from decimal import Decimal
import benchmark_functions as bf
import scipy

class Chromosom:
    def __init__(self, precision, variables_count, start_, end_):
        self.precision = precision
        self.variables_count = variables_count
        self.start_ = start_
        self.end_ = end_
        self.chromosom_len = ceil(self.precision * log2(self.end_ - self.start_))
        self.chromosoms = self._generate_chromosom()
        self.decoded_chromosom = self._decode_chromosom()

    def _generate_chromosom(self) -> list:
      chromosoms = []
      for i in range(self.variables_count):
        chromosom = []
        for i in range(self.chromosom_len):
            chromosom.append(random.randint(0, 1))
        chromosoms.append(chromosom)
      return chromosoms

    def _decode_chromosom(self) -> list:
      decoded_chromosom = []
      for chromosom in self.chromosoms:
        decimal_number = sum(bit * (2 ** i) for i, bit in enumerate(reversed(chromosom)))
        decoded = self.start_ + decimal_number * (self.end_ - self.start_) / (2 ** self.chromosom_len - 1)
        decoded_chromosom.append(Decimal(decoded).quantize(Decimal(10) ** -self.precision))
      return decoded_chromosom
    
    def __str__(self):
        return f"Chromosoms: {self.chromosoms} | Value in Decimal: {self.decoded_chromosom}"


class Individual:
    def __init__(self, precision, variables_count, start_, end_):
        self.chromosom = Chromosom(precision, variables_count, start_, end_)
        self.variables_count = variables_count

    def __str__(self):
        return f"{self.chromosom}"


class Population:
    def __init__(self, variables_count, population_size, precision, start_, end_, func, optimum):
        self.variables_count = variables_count
        self.population_size = population_size
        self.func = func 
        self.individuals = [Individual(precision, variables_count, start_, end_) for _ in range(self.population_size)]
        self.optimum = 0 if optimum == "min" else 1
        self.cell = self.getCell()
        self.precision = precision

    def getCell(self) -> dict:
        """Oblicza wartości funkcji celu dla populacji."""
        cell_dict = {}
        X = self.getX()
        for x in X:
            x_tuple = tuple(x)  
            cell_value = self.func(x) 
            cell_dict[x_tuple] = cell_value  
        return cell_dict
        
    def getSortedCell(self) -> list[tuple[tuple[float], float]]:
        """Sortuje wartości funkcji celu."""
        return sorted(self.cell.items(), key=lambda item: item[1], reverse=self.optimum)

    def getBestBySelection(self, percentage: float,) -> list:
        """Zwraca najlepsze osobniki według funkcji celu."""
        size = int(percentage * self.population_size / 100)
        best_individuals = self.getSortedCell()
        return best_individuals[:size]
    
    def getBestByTournament(self, k) -> list:
      """Zwraca najlepsze osobniki według funkcji celu."""
      if k > self.population_size:
          raise ValueError("k parameter cannot be greater than population_size!")
      
      cell = self.cell.copy()
      best_individuals = []

      for _ in range(k):
          tournament = random.sample(list(cell.items()), int(self.population_size / k))  # Losujemy podzbiór
          if self.optimum == 1:
            best = max(tournament, key=lambda x: x[1])  # Wybieramy najlepszego osobnika (dla maksymalizacji)
          else:
            best = min(tournament, key=lambda x: x[1])  # Wybieramy najlepszego osobnika (dla minimalizacji)
          best_individuals.append(best)
          del cell[best[0]]  # Usuwamy wybranego osobnika

      return best_individuals

    def getBestByRulet(self) -> list:
        """Zwraca najlepsze osobniki według funkcji celu."""
        cell = self.cell.copy()
        best_individuals = []
        if self.optimum == 0:
          cell = {key: 1/value for key, value in cell.items()}
        total_fitness = sum(cell.values())
        if total_fitness == 0:
            raise ValueError("Total fitness is zero!")

        probabilities = {}
        distribution = {}
        distribution_value = 0

        for value in cell.values():
            probability = value / total_fitness
            probabilities[value] = probability
            distribution_value += probability
            distribution[value] = distribution_value

        if round(sum(probabilities.values())) != 1.0:
            raise ValueError("Sum of probabilities is not equal to 1!")
        last_item = round(list(distribution.items())[-1][1])
        if last_item != 1.0:
            raise ValueError("Last distribution value is not equal to 1!")
        num = random.random()
        for key, value in distribution.items():
            if num <= value:
                x = next(k for k, v in cell.items() if v == key)
                best_individuals.append((x, self.cell[x]))  # Zapisujemy klucz + wartość
                del cell[x]  # Usuwamy osobnika
                break
        return best_individuals

    def getX(self) -> list:
        """Zwraca listę fenotypów (wartości zmiennych)."""
        return [[float(x) for x in individual.chromosom.decoded_chromosom] for individual in self.individuals]

    def __str__(self):
        return "\n".join(str(individual) for individual in self.individuals)



In [34]:
c = Chromosom(10, 5, -10, 10)
chromosoms = c.chromosoms
decoded_chromosom = c.decoded_chromosom
print(c)


Chromosoms: [[1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0], [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]] | Value in Decimal: [Decimal('2.1306477600'), Decimal('-5.7195073087'), Decimal('-2.3093512589'), Decimal('3.3059095214'), Decimal('-8.3043134744')]


In [3]:
func = bf.Schwefel(n_dimensions=3)

population = Population(3, 10, 10, -10, 10, func, "min")

population.getBestByRulet()


[((1.102796058, 6.2400302594, 7.6053796236), 1249.3997214449325)]